In [ ]:
%matplotlib widget
# %matplotlib inline

In [ ]:
from pathlib import Path
import re

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from fluidsim import load
from fluidsim.util import get_dataframe_from_paths

In [ ]:
height = 5.5
plt.rc("figure", figsize=(1.33 * height, height))

In [ ]:
path_base = Path("/fsnet/project/meige/2022/22STRATURBANIS/aniso")

def get_path_finer_resol(N, Rb):
    paths = sorted(path_base.glob(f"ns3d.strat*_Rb{Rb}*_N{N}*"))
    return paths[-1]

paths320 = sorted(path_base.glob(f"ns3d.strat*_320x320*"))
couples = set()
for path in paths320:
    name = path.name
    Rb = re.search(r"_Rb(.*?)_", path.name).group(1)
    N = re.search(r"_N(.*?)_", path.name).group(1)
    couples.add((N, Rb))
    
paths = []
for N, Rb in couples:
    paths.append(get_path_finer_resol(N, Rb))
        
[p.name for p in paths]

In [ ]:
def customize(result, sim):
    result["Rb"] = float(sim.params.short_name_type_run.split("_Rb")[-1])
    result["nx"] = sim.params.oper.nx 


df = get_dataframe_from_paths(paths, tmin="t_start+1", use_cache=True, customize=customize)

In [ ]:
df

In [ ]:
df[(df.Fh < 0.04) & (df.Fh > 0.02) & (df.R4 > 8)]

In [ ]:
def plot(df, x, y, logx=True, logy=False, c=None, vmin=None, vmax=None):
    ax = df.plot.scatter(
        x=x, y=y, logx=logx, logy=logy, c=c, edgecolors="k", vmin=vmin, vmax=vmax
    )
    pc = ax.collections[0]
    pc.set_cmap("viridis")
    plt.colorbar(pc, ax=ax)
    return ax

In [ ]:
plot(df, "Fh", "R2", c=df.I_velocity)

In [ ]:
plot(df, "Fh", "R2", c=df.I_dissipation)

In [ ]:
plot(df, "Fh", "R4", c=df.I_velocity, logy=True)

In [ ]:
plot(df, "Fh", "R2", c=df["epsK2/epsK"], logy=True)

In [ ]:
plot(df, "Fh", "epsK2/epsK", c=np.log10(df["R2"]))

In [ ]:
plot(df, "Fh", "I_velocity", c=np.log10(df["R2"]))

In [ ]:
plot(df, "R2", "I_dissipation", c=np.log10(df["R2"]))

In [ ]:
def plot2(dataframe, ax=None, color=None):
    return dataframe.plot.scatter(
        x="Fh", y="Gamma", logx=True, ax=ax, color=color
    )


tab10 = plt.get_cmap("tab10")

Ns = sorted(df.N.unique())

ax = None
for iN, N in enumerate(Ns):
    ax = plot2(df[df.N == N], ax=ax, color=tab10(iN))


fig = ax.figure

ax_sub = fig.add_axes([0.6, 0.6, 1.33 * 0.2, 0.2])


def plot2(dataframe, color=None):
    dataframe.plot.scatter(
        x="Fh", y="Gamma", logx=True, logy=True, ax=ax_sub, color=color
    )


for iN, N in enumerate(Ns):
    ax = plot2(df[df.N == N], color=tab10(iN))

ax_sub.set_xlabel("")
ax_sub.set_ylabel("")

xs = np.linspace(7e-2, 3.5e-1, 4)
ax_sub.plot(xs, 3e-2 * xs**-1)
ax_sub.text(0.1, 0.1, "$k^{-1}$")

xs = np.linspace(5e-1, 1.5, 4)
ax_sub.plot(xs, 3e-2 * xs**-2)
ax_sub.text(1, 0.05, "$k^{-2}$");

In [ ]:
plot(df, "Fh", "Gamma", c=np.log10(df["R2"]), vmin=0, vmax=2)

In [ ]:
plot(df, "Fh", "Gamma", c=np.log10(df["min_R"]), vmin=0, vmax=2)